## What is Support Vector Machine Algorithm?
- Powerful ML for regression and classification tasks, but best suited for classification
- Tries to find the maximum separating hyperplane between the different classes available in the target
- Finding the optimal hyperplane in an N-dimensional space that can separate data points in different classes in feature space.
<div>
<img src="svm_hyperplane.png" width="300">
</div>
- 2-D plane if you have three features.
- SVM is similar to decision tree that treis to find a best split, but rather focused on hyperplane in more than one feature.
- The hyperplane treis that the margin between closest points of different classes should be as max as possible. 
- SVM works the best when you think you have data that can be segregated, but their relationships are not linear. ex) 

### 1. How does SVM work?
- Choose the hyperplane that has the nearest data point on each side is maximized. (Similar to linear regression fitted line)
- Hyperplane can ignore outliers
- SVM tries to minimize $ \frac{1}{margin} + (\sum penalty) $ where margin is the distance between closest datapoints from each class
- Hinge loss is commonly used.

### 2. Terminology of SVM
1) <b>Hyperplance</b> : linear equation - wx+b = 0
- $ w^Tx + b = 0 $
- $ d_i = \frac{w^Tx_i+b}{||w||} $
- $ f(x) = 
\begin{cases} 
1 : w^Tx + b >=0 \\
0 : w^Tx + b <0
\end{cases} $

2) <b>Weight Vectors</b> : defines the orientation of hyperplane separating the classes.

3) <b>Support Vectors</b> : closest data points to the hyperplane
  
4) <b>Margin</b> : distance between sv and hyperplane
  
5) <b>kernel</b> : math function used to map the original input data points into high-dimensional feature spaces. It helps to find the best hyperplane if the data points are not linearly separable in the original input space. Fns are linear, polynomial, radial basis, and sigmoid.
  
6) <b>Hard Margin</b> : Separates the data points of each class without any misclassification
- $ minimize_{w,b} \frac{1}{2}w^Tw = minimize_{w,b} \frac{1}{2} ||w||^2 $
- subject to $ y_i(w^Tx_i + b)>= 1 $ for i = 1,2,3,...,m




7) <b>Soft Margin</b> : This technique helps when the data is not perfectly separable by adopting slack variable in each data point. This slack variable smoothens the margin requirement and allow some violations. There is a balance trade off between increasing margin and reducing violation.
- $ \min_{w, b, \xi} \frac{1}{2} \|w\|^2 + C \sum_{i=1}^{n} \xi_i $
- $ \text{subject to:} y_i (w \cdot x_i + b) \geq 1 - \xi_i, \forall i $
- $ \xi_i \geq 0, \forall i $

When it cannot be spliited because it is not linear.
<div>
<img src="svm_hard_case.png" width="300">
</div>

Kernel tricks can split the data.
<div>
<img src="svm_kernel_applied.png" width="300">
</div>



8) <b>C</b> : Regularization parameter C is a penalty term that stricts between misclassification and margin.
9) <b>Hinge Loss</b> : penlizes the misclassification and points that are correctly classified, but too close to the decision boundary.
10) <b>Dual Problem</b> : Optimization problem - Lagrange multipliers to solve SVM. Dual formulation enables to use kernel tricks and effective computing.
- Here $\alpha_i$ is Lagrange multiplier to find the local min/max and associated with the data points representing importance of each data points.
- $
\max_{\alpha} \sum_{i=1}^{n} \alpha_i - \frac{1}{2} \sum_{i=1}^{n} \sum_{j=1}^{n} \alpha_i \alpha_j y_i y_j K(x_i, x_j)
$
- $
\text{subject to:} 0 \leq \alpha_i \leq C, \quad \forall i
$
- $
\sum_{i=1}^{n} \alpha_i y_i = 0
$
- $ f(x) = \sum_{i=1}^{n} \alpha_i y_i K(x_i, x) + b $

### 3. Procedure
1. Normalize/standardize
2. Select model type either hard-margin/soft-margin
3. Choose kernel for non-linear classification; liner, polynomial, RBF(Gaussian)
4. Optimization problem to find the best optimal hyperplane
5. Make prediction

### 4. Advanatages over other modesl
1. Effective in high-dimensional cases.
2. Its memory is efficient as it uses a subset of training points in the decision function called support vectors.
3. Different kernel functions can be specified for the decision functions and its possible to specify custom kernels.
4. Well performing for high dimentional cases and disadvantages for large dataset
- Reference Link : https://www.geeksforgeeks.org/support-vector-machine-algorithm/

In [66]:
import numpy as np
from cvxopt import matrix, solvers  # Used for quadratic programming

# Define the polynomial kernel function
def polynomial_kernel(x1, x2, degree=3, coef0=1):
    return (np.dot(x1, x2) + coef0) ** degree

class SVM:
    def __init__(self, C=1.0, kernel=polynomial_kernel, degree=3, coef0=1):
        self.C = C  # Regularization parameter
        self.kernel = kernel  # Kernel function
        self.degree = degree  # Degree of polynomial kernel
        self.coef0 = coef0  # Coefficient for polynomial kernel

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Apply Kernel function
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j], self.degree, self.coef0)
        
        # Construct the matrices for the quadratic programming problem
        #if y is (n,1) np.outer makes it (n,n) outer product
        #(1,2,3) outer product (1,2,3) = ([1,2,3],[2,4,6],[3,6,9])
        P = matrix(np.outer(y, y) * K) 
        q = matrix(-np.ones(n_samples))

        #np.eye is diagonal matrix
        #vstack concatenate matrix vertically
        #hstack concatenate matrix vertically
        G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
        h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C)))
        #A is y into matrix with 1xn
        A = matrix(y, (1, n_samples), 'd')
        b = matrix(0.0)

        # Solve the quadratic programming problem using cvxopt
        solvers.options['show_progress'] = False

        #b is 0 on the right hand side
        #P is kernal matrix
        # q is -1 vector
        # G is block matrix combination of identity matrices and negative identity matrices to handle contraints
        # h is for upper and lower bound using C (Penalty regularization term in soft margin)
        # A is a label vector (1xn)
        solution = solvers.qp(P, q, G, h, A, b)
        #Flattens the result matrix into 1d
        alphas = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers
        #This masking operation filters out very small values of alpha
        #1e-5 is used to select only effective data points to be considered in support vectors
        sv = alphas > 1e-5
        self.alphas = alphas[sv]
        self.support_vectors_ = X[sv]
        self.sv_y = y[sv]

        # Weight vector calculation
        self.w = np.sum(self.alphas[:, None] * self.sv_y[:, None] * self.support_vectors_, axis=0)

        # Calculate bias
        self.b = np.mean([y_k - np.sum(self.alphas * self.sv_y * K[sv_idx, sv])
                          for sv_idx, y_k in zip(np.where(sv)[0], self.sv_y)])

    def predict(self, X):
        y_pred = np.sum(self.alphas * self.sv_y * np.array([self.kernel(sv, X, self.degree, self.coef0) for sv in self.support_vectors_]), axis=0) + self.b
        return np.sign(y_pred)

# Preprocessing Iris dataset
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:100, :2]  # Only two features for visualization, and two classes for binary classification
y = iris.target[:100]
y = np.where(y == 0, -1, 1)  # Convert to -1 and 1

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Train the SVM model with a polynomial kernel
svm = SVM(C=1.0, kernel=polynomial_kernel, degree=3, coef0=1)
svm.fit(X_train, y_train)

# Predict and evaluate
y_pred = np.array([svm.predict(x) for x in X_test])
accuracy = np.mean(y_pred == y_test)

print("Accuracy:", accuracy)

Accuracy: 1.0


In [64]:
np.outer(y_train,y_train)[:,None]

array([[[ 1,  1,  1, ...,  1, -1, -1]],

       [[ 1,  1,  1, ...,  1, -1, -1]],

       [[ 1,  1,  1, ...,  1, -1, -1]],

       ...,

       [[ 1,  1,  1, ...,  1, -1, -1]],

       [[-1, -1, -1, ..., -1,  1,  1]],

       [[-1, -1, -1, ..., -1,  1,  1]]])

In [34]:
s_shape0 = X_train.shape[0]
K = np.zeros((s_shape0,s_shape0))
for i in range(s_shape0):
    for j in range(s_shape0):
        K[i,j] = polynomial_kernel(X_train[i], X_train[j], 3, 1)

In [35]:
K.shape

(60, 60)

In [36]:
np.dot(y_train,y_train)

60

In [40]:
matrix(np.outer(y_train,y_train)*K)

<60x60 matrix, tc='d'>

In [54]:
np.ravel(np.vstack((np.eye(2),np.eye(2))))

array([1., 0., 0., 1., 1., 0., 0., 1.])

In [53]:
matrix(y_train, (1, 60), 'd')



<1x60 matrix, tc='d'>

In [59]:
import numpy as np
from cvxopt import matrix, solvers

# Define the quadratic and linear parts of the objective function
P = matrix([[1.0, 0.0], [1.0, 1.0]])  # Coefficients for x1^2 and x2^2
q = matrix([0.0, 1.0])  # Coefficients for linear terms (none in this case)

# No constraints: no G, h, A, b

# Solve the quadratic programming problem
solution = solvers.qp(P, q)

# Extract the optimal solution
x_opt = np.array(solution['x'])
print("Optimal solution:", x_opt)

Optimal solution: [[-0.]
 [-1.]]


In [62]:
1*1e-5

0